### There are two entities: **Alice and Bob,** to do five main steps.

### **FIVE main steps**

1. **Alice** <u>creates </u>the required parameters for DH key exchange, creates her own private and public keys, and sends the public parameters and her public keys to Bob
2. **Bob** creates his own public key, and sends it to Alice. Bob also creates a derived shared key \(to be used later for AES\-128 encryption\)
3. **Alice** received Bob's public key and also creates a derived shared key \(to be used later for AES\-128 encryption\)
4. **Bob** <u>reads </u>the content of message.txt and <u>sends </u>the following to **Alice**:
   - The content of message.txt encrypted using the shared key \(generated in step 2\)
   - The \(SHA256\) hash value of the message
5. **Alice** <u>receives </u>the set of information from **Bob**, and <u>does </u>the following:
   1. Decrypt the hash value of the message, 
   2. Decrypt the message using the shared key \(generated in step 3\)
   3. Hash \(the received and decrypted\) message and checks which it is identical to \(the received\) hash value.

**Your task is to complete the below code to implement the above task.**



## Step 1: Alice generates DH parameters



In [3]:
#
# WRITE your supportive code here
#
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
import secrets
from hashlib import sha256
from Crypto.Cipher import AES

parameters = dh.generate_parameters(generator=2, key_size=512)

In [4]:
def step_1():
    #
    #
    # WRITE your main code here
    #
    alice_x = parameters.generate_private_key()
    alice_y = alice_x.public_key()
    p = parameters.parameter_numbers().p
    g = parameters.parameter_numbers().g
    # id alice_y is private_key should we return it. In the task it asks to return piblic keys??
    return alice_x, alice_y, p, g

In [5]:
alice_x, alice_y, p, g = step_1()

## Step 2: Bob generates DH parameters



In [6]:
    #
    # WRITE your supportive code here
    #

In [7]:
def step_2(p, g, alice_y):
    #
    # WRITE your main code here
    #
    bob_x = parameters.generate_private_key()
    bob_y = bob_x.public_key()
    bob_shared_key = bob_x.exchange(alice_y)

    bob_derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(bob_shared_key)

    return bob_derived_key, bob_y

In [8]:
bob_derived_key, bob_y = step_2(p, g, alice_y)

## Step 3:  Alice generates final key



In [9]:
    #
    # WRITE your supportive code here
    #

In [10]:
def step_3(alice_x, p, g, alice_y, bob_y):
    #
    # WRITE your main code here
    #
    alice_shared_key = alice_x.exchange(bob_y)
    alice_derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
    ).derive(alice_shared_key)

    return alice_derived_key

In [19]:
alice_derived_key = step_3(alice_x, p, g, alice_y, bob_y)


True


## Step 4: Bob sends a message to Alice



In [20]:
#
# WRITE your supportive code here
#
#PADDING
BLOCK_SIZE = 16 #bytes
pad = lambda s: s + (BLOCK_SIZE - len(s)%BLOCK_SIZE) * chr(BLOCK_SIZE - len(s) % BLOCK_SIZE)
unpad = lambda s: s[:-ord(s[len(s) - 1:])]

# Bob reads the content of message.txt and sends the following to Alice:
# The content of message.txt encrypted using the shared key (generated in step 2)
# The (SHA256) hash value of the message

In [21]:
def step_4(bob_derived_key):
    #
    # WRITE your main code here
    #
    with open('message.txt', 'r') as file:
        message = file.read()
    original_message_hash = sha256(message.encode()).hexdigest()

    iv = secrets.token_bytes(BLOCK_SIZE)
    Enc = AES.new(bob_derived_key, AES.MODE_CBC, iv)
    data = pad(message).encode()
    ciphertext = Enc.encrypt(data)
    ciphertext_hex = iv.hex() + ciphertext.hex()

    return ciphertext_hex, original_message_hash

In [22]:
cipher, original_message_hash = step_4(bob_derived_key)

True
True


## Step 5: Alice receives the message from Bob



In [15]:
#
# WRITE your supportive code here
#
# Decrypt the message using the shared key (generated in step 3)
# Hash (the received and decrypted) message and checks which it is identical to (the received) hash value.


In [25]:
def step_5(alice_derived_key, cipher, original_message_hash):
    #
    # WRITE your main code here
    #
    iv = bytes.fromhex(cipher[:32])
    ciphertext = bytes.fromhex(cipher[32:])
    DEC = AES.new(alice_derived_key, AES.MODE_CBC, iv)
    pt = DEC.decrypt(ciphertext)
    unpadded_pt = unpad(pt)
    print(unpadded_pt.decode())
    decrypted_message = unpadded_pt.decode()
    decrypted_message_hash = sha256(unpadded_pt).hexdigest()

    # decrypted_message is the decrypted cipher, and result is a Boolean True/False variable.
    return decrypted_message, original_message_hash==decrypted_message_hash

In [26]:
print(step_5(alice_derived_key, cipher, original_message_hash)) # The output should be the message along with the verification result (true in this case)

this is a secret message from Bob to Alice
('this is a secret message from Bob to Alice', True)
